In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load the Montreal borough shapefile
borough_shapefile = 'limites-administratives-agglomeration-nad83.shp'  
boroughs = gpd.read_file(borough_shapefile)

# Load EV charging station data
ev_stations_file = 'filtered_ev_chagers.csv'  
ev_stations = pd.read_csv(ev_stations_file)

# Convert EV charging stations to GeoDataFrame with Point geometry
ev_stations['geometry'] = ev_stations.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
ev_gdf = gpd.GeoDataFrame(ev_stations, geometry='geometry')

# Set the coordinate reference system (CRS) for both datasets (same CRS for spatial join)
ev_gdf = ev_gdf.set_crs('EPSG:4326')

# Transform the boroughs CRS to EPSG:4326
boroughs = boroughs.to_crs('EPSG:4326')

# Step 5: Perform a spatial join to assign EV stations to boroughs
ev_with_boroughs = gpd.sjoin(ev_gdf, boroughs, how="left", predicate='within')

# Load air quality data 
air_quality_file = 'filtered_rsqa-indice-qualite-air-station.csv'  
air_quality = pd.read_csv(air_quality_file)

# Convert air quality data to GeoDataFrame with Point geometry
air_quality['geometry'] = air_quality.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
air_quality_gdf = gpd.GeoDataFrame(air_quality, geometry='geometry')

# Ensure CRS alignment for air quality data
air_quality_gdf = air_quality_gdf.set_crs('EPSG:4326', allow_override=True)

# Step 8: Perform the spatial join to assign air quality measurements to boroughs
air_quality_with_boroughs = gpd.sjoin(air_quality_gdf, boroughs, how="left", predicate='within')

# Check results
# Update the column name for the boroughs after spatial join
# Assuming 'NOM' is the borough column in the spatial join result
print(ev_with_boroughs[['NOM_BORNE_RECHARGE', 'NOM_PARC', 'ADRESSE', 'VILLE', 'PROVINCE', 'NIVEAU_RECHARGE', 'NOM']])
print(air_quality_with_boroughs[['stationId', 'adresse', 'polluant', 'valeur', 'date', 'heure', 'NOM']])

# Optionally, save the results to CSV
ev_with_boroughs.to_csv('ev_stations_with_boroughs.csv', index=False)
air_quality_with_boroughs.to_csv('air_quality_with_boroughs.csv', index=False)
